In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000
adapt_c = True

calc_c =lambda nobs: (nobs**(1/4)/np.sqrt(nobs), .2*nobs**(1/3))

# a  = .25, k= 9

In [5]:
a = 0.25
num_params=9



In [6]:
nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.94 & 0.95 & 0.88 & 0.98 & 0.97   \\
Model 1 & 0.06 & 0.06 & 0.05 & 0.12 & 0.02 & 0.03   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.942, 0.058, 0.   ]), array([0.942, 0.058, 0.   ]), array([0.948, 0.052, 0.   ]), array([0.88, 0.12, 0.  ]), array([0.982, 0.018, 0.   ]), array([0.974, 0.026, 0.   ]), 3.0674708491603426, 4.819876674965241, 4.88089001419416)


In [7]:
nobs = 500
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.88 & 0.88 & 0.88 & 0.80 & 0.96 & 0.91   \\
Model 1 & 0.12 & 0.12 & 0.12 & 0.20 & 0.04 & 0.09   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.876, 0.124, 0.   ]), array([0.876, 0.124, 0.   ]), array([0.88, 0.12, 0.  ]), array([0.802, 0.198, 0.   ]), array([0.958, 0.042, 0.   ]), array([0.914, 0.086, 0.   ]), 6.462326840241201, 6.326269005084871, 6.678905093353744)


In [8]:
nobs = 100
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.98 & 0.99 & 0.97 & 1.00 & 1.00   \\
Model 1 & 0.01 & 0.00 & 0.01 & 0.03 & 0.00 & 0.00   \\
Model 2 & 0.01 & 0.01 & 0.01 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.978, 0.01 , 0.012]), array([0.984, 0.004, 0.012]), array([0.986, 0.008, 0.006]), array([0.968, 0.03 , 0.002]), array([0.998, 0.002, 0.   ]), array([0.996, 0.004, 0.   ]), -0.025085600241920237, 3.2433773223803617, 3.4049794372495223)


# a  = .25, k= 4, something not right?

In [9]:
a = 0.25
num_params=4



In [10]:

nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.94 & 0.95 & 0.97 & 0.99 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.05 & 0.05 & 0.04 & 0.02 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.94, 0.01, 0.05]), array([0.94, 0.01, 0.05]), array([0.952, 0.008, 0.04 ]), array([0.968, 0.014, 0.018]), array([0.992, 0.002, 0.006]), array([0.988, 0.002, 0.01 ]), -2.0138734514716634, 5.65176236004376, 5.595450111705784)


# a  = .25, K = 19

In [11]:
a = 0.25
num_params=19


In [12]:

nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.90 & 0.90 & 0.91 & 0.82 & 0.98 & 0.97   \\
Model 1 & 0.10 & 0.10 & 0.09 & 0.18 & 0.02 & 0.03   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.902, 0.096, 0.002]), array([0.902, 0.096, 0.002]), array([0.906, 0.094, 0.   ]), array([0.818, 0.182, 0.   ]), array([0.976, 0.024, 0.   ]), array([0.966, 0.034, 0.   ]), 3.9425700081378383, 4.423597787377051, 4.497060309519261)


# evidence of power

In [13]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [14]:

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.55 & 0.55 & 0.60 & 0.41 & 0.79 & 0.71   \\
Model 1 & 0.45 & 0.45 & 0.40 & 0.59 & 0.21 & 0.29   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.55, 0.45, 0.  ]), array([0.55, 0.45, 0.  ]), array([0.598, 0.402, 0.   ]), array([0.406, 0.594, 0.   ]), array([0.794, 0.206, 0.   ]), array([0.71, 0.29, 0.  ]), 9.265474553159423, 4.932896696542937, 4.9530555836449475)


In [15]:

nobs = 500
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.55 & 0.55 & 0.57 & 0.38 & 0.84 & 0.72   \\
Model 1 & 0.45 & 0.45 & 0.43 & 0.62 & 0.16 & 0.28   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.546, 0.454, 0.   ]), array([0.546, 0.454, 0.   ]), array([0.572, 0.428, 0.   ]), array([0.382, 0.618, 0.   ]), array([0.844, 0.156, 0.   ]), array([0.716, 0.284, 0.   ]), 9.575537751452938, 4.675572305724983, 5.02272887860208)


In [16]:

nobs = 100
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.53 & 0.54 & 0.65 & 0.49 & 0.72 & 0.72   \\
Model 1 & 0.47 & 0.46 & 0.35 & 0.51 & 0.28 & 0.28   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.532, 0.468, 0.   ]), array([0.544, 0.456, 0.   ]), array([0.652, 0.348, 0.   ]), array([0.49, 0.51, 0.  ]), array([0.722, 0.278, 0.   ]), array([0.724, 0.276, 0.   ]), 9.217969603861947, 4.71954767157488, 4.730679407816759)


In [17]:

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.60 & 0.60 & 0.65 & 0.46 & 0.82 & 0.75   \\
Model 1 & 0.40 & 0.40 & 0.35 & 0.54 & 0.18 & 0.25   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.602, 0.398, 0.   ]), array([0.602, 0.398, 0.   ]), array([0.646, 0.354, 0.   ]), array([0.464, 0.536, 0.   ]), array([0.818, 0.182, 0.   ]), array([0.746, 0.254, 0.   ]), 9.104906998874913, 4.710429924138607, 4.929491898122051)


In [18]:

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.58 & 0.58 & 0.61 & 0.42 & 0.81 & 0.74   \\
Model 1 & 0.42 & 0.42 & 0.39 & 0.58 & 0.19 & 0.26   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.584, 0.416, 0.   ]), array([0.584, 0.416, 0.   ]), array([0.608, 0.392, 0.   ]), array([0.424, 0.576, 0.   ]), array([0.806, 0.194, 0.   ]), array([0.736, 0.264, 0.   ]), 8.990275746793133, 4.547286241831195, 4.857412622814009)


# evidence of power 2

In [19]:
nobs = 250
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.72 & 0.73 & 0.75 & 0.91 & 0.99 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.28 & 0.27 & 0.25 & 0.09 & 0.01 & 0.02   \\
\hline
\end{tabular}
(array([0.722, 0.   , 0.278]), array([0.726, 0.   , 0.274]), array([0.75, 0.  , 0.25]), array([0.91, 0.  , 0.09]), array([0.992, 0.   , 0.008]), array([0.976, 0.   , 0.024]), -6.270732831503361, 3.3133061042024865, 3.638486314214882)


In [20]:
nobs = 500
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.76 & 0.76 & 0.78 & 0.91 & 1.00 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.24 & 0.24 & 0.22 & 0.09 & 0.00 & 0.01   \\
\hline
\end{tabular}
(array([0.756, 0.   , 0.244]), array([0.76, 0.  , 0.24]), array([0.776, 0.   , 0.224]), array([0.91, 0.  , 0.09]), array([0.996, 0.   , 0.004]), array([0.988, 0.   , 0.012]), -6.15937431963855, 3.2636316360892903, 3.644748507005801)


In [21]:
nobs = 100
c1,c2 = calc_c(nobs)



a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.73 & 0.78 & 0.80 & 0.92 & 0.98 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.27 & 0.22 & 0.20 & 0.08 & 0.02 & 0.02   \\
\hline
\end{tabular}
(array([0.728, 0.   , 0.272]), array([0.78, 0.  , 0.22]), array([0.8, 0. , 0.2]), array([0.916, 0.   , 0.084]), array([0.978, 0.   , 0.022]), array([0.984, 0.   , 0.016]), -5.789555662057122, 3.3765933129739625, 3.3722803294615322)


In [22]:
nobs = 250
c1,c2 = calc_c(nobs)


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.19 & 0.20 & 0.22 & 0.42 & 0.80 & 0.73   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.81 & 0.80 & 0.78 & 0.58 & 0.20 & 0.27   \\
\hline
\end{tabular}
(array([0.19, 0.  , 0.81]), array([0.196, 0.   , 0.804]), array([0.222, 0.   , 0.778]), array([0.422, 0.   , 0.578]), array([0.8, 0. , 0.2]), array([0.732, 0.   , 0.268]), -12.450487088189513, 4.776471734482983, 4.901841181522841)


In [23]:
nobs = 250
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=False)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.94 & 0.98 & 1.00 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.07 & 0.07 & 0.06 & 0.02 & 0.00 & 0.01   \\
\hline
\end{tabular}
(array([0.926, 0.   , 0.074]), array([0.93, 0.  , 0.07]), array([0.94, 0.  , 0.06]), array([0.978, 0.002, 0.02 ]), array([0.996, 0.   , 0.004]), array([0.994, 0.   , 0.006]), -3.8983809573555246, 2.6418303677741712, 2.946622009224848)
